In [2]:
from Code.PK_model import ChiPKLin
import numpy as np
import chi.library
# Define 1-compartment pharmacokinetic model
mechanistic_model = ChiPKLin(num_comp=2)
# mechanistic_model = chi.library.ModelLibrary().one_compartment_pk_model()
print("parameters:", mechanistic_model.parameters())

# Define parameters
PK_params = np.load("../Data_and_parameters/PK_sim/actual_params.npy")
PK_params = PK_params[:, [1,0,3,2, -1]]
PK_param_names = PK_params[0, :]
PK_params = PK_params[1, :].astype('float64')
print(PK_param_names)
# parameters = [
#     10,  # Initial drug amount
#     1,   # Elimination rate
#     2,   # Volume of the compartment
#     1,   # Elimination rate
#     2,   # Volume of the compartment
# ]

# Define evaluation times
times =  [0, 0.2, 0.5, 0.6, 1]

# Simulate the model
result = mechanistic_model.simulate(PK_params[:-1], times)
result

INFO:root:running build_ext
INFO:root:building 'myokit_sim_2_3684731453083825083' extension
INFO:root:creating build
INFO:root:creating build/temp.linux-x86_64-cpython-310
INFO:root:creating build/temp.linux-x86_64-cpython-310/tmp
INFO:root:creating build/temp.linux-x86_64-cpython-310/tmp/tmp245xz3ztmyokit
INFO:root:x86_64-linux-gnu-gcc -Wno-unused-result -Wsign-compare -DNDEBUG -g -fwrapv -O2 -Wall -g -fstack-protector-strong -Wformat -Werror=format-security -g -fwrapv -O2 -fPIC -I/usr/local/include -I/opt/local/include -I/home/rumney/Documents/Myleotoxicity/Myleotoxicity-PKPD/venv/lib/python3.10/site-packages/myokit/_sim -I/home/rumney/Documents/Myleotoxicity/Myleotoxicity-PKPD/venv/lib/python3.10/site-packages/myokit/_sim -I/home/rumney/Documents/Myleotoxicity/Myleotoxicity-PKPD/venv/include -I/usr/include/python3.10 -c /tmp/tmp245xz3ztmyokit/source.c -o build/temp.linux-x86_64-cpython-310/tmp/tmp245xz3ztmyokit/source.o
INFO:root:creating build/lib.linux-x86_64-cpython-310
INFO:root

parameters: ['central.K_cl', 'central.V_c', 'periferal_1.K_p', 'periferal_1.V_p']
['K_{cl}, L/hr' 'V_c, L' 'K_1, L/hr' 'V_1, L' '\\sigma_{m, PK}']


array([[0., 0., 0., 0., 0.]])

In [11]:
import numpy as np
import plotly.graph_objects as go

dose_amts = [0.0, 1.0, 2.0, 3.0]
times = np.linspace(start=0, stop=5, num=1000)
dose_unit =  "mg"
fig = go.Figure()

for dose in dose_amts:
    # Set administration and dosing regimen
    mechanistic_model.set_administration(
        compartment='central', amount_var='drug_amount')
    mechanistic_model.set_dosing_regimen(dose=dose, period=0)

    # Simulate the model
    result = mechanistic_model.simulate(PK_params[:-1], times)
    
    # Visualise results
    fig.add_trace(go.Scatter(
        x=times,
        y=result[0],
        name="Dose "+str(dose)+" "+dose_unit,
        mode='lines'
    ))
fig.update_layout(
    xaxis_title='Time',
    yaxis_title='Drug concentration',
    template='plotly_white'
)
fig.update_yaxes(type="log", dtick=1)
fig.show()

INFO:root:running build_ext
INFO:root:building 'myokit_sim_16_3684731453083825083' extension
INFO:root:creating build
INFO:root:creating build/temp.linux-x86_64-cpython-310
INFO:root:creating build/temp.linux-x86_64-cpython-310/tmp
INFO:root:creating build/temp.linux-x86_64-cpython-310/tmp/tmpp_ammul2myokit
INFO:root:x86_64-linux-gnu-gcc -Wno-unused-result -Wsign-compare -DNDEBUG -g -fwrapv -O2 -Wall -g -fstack-protector-strong -Wformat -Werror=format-security -g -fwrapv -O2 -fPIC -I/usr/local/include -I/opt/local/include -I/home/rumney/Documents/Myleotoxicity/Myleotoxicity-PKPD/venv/lib/python3.10/site-packages/myokit/_sim -I/home/rumney/Documents/Myleotoxicity/Myleotoxicity-PKPD/venv/lib/python3.10/site-packages/myokit/_sim -I/home/rumney/Documents/Myleotoxicity/Myleotoxicity-PKPD/venv/include -I/usr/include/python3.10 -c /tmp/tmpp_ammul2myokit/source.c -o build/temp.linux-x86_64-cpython-310/tmp/tmpp_ammul2myokit/source.o
INFO:root:creating build/lib.linux-x86_64-cpython-310
INFO:roo

In [22]:
import chi

# Define error model and error model parameters
error_model = chi.MultiplicativeGaussianErrorModel()

# Down sample times and mechanistic model evaluations
measurement_times = times[12::25]
corresponding_evaluations = result[0, 12::25]

# Simulate measurements
measurements = error_model.sample(
    [PK_params[-1]], model_output=corresponding_evaluations, seed=1)[:, 0]

# Visualise results
fig = go.Figure()
fig.add_trace(go.Scatter(
    x=times,
    y=result[0],
    mode='lines',
    line_color='black',
    name='Mechanistic model'
))
fig.add_trace(go.Scatter(
    x=measurement_times,
    y=measurements,
    mode='markers',
    name='Measurements'
))
fig.update_layout(
    xaxis_title='Time',
    yaxis_title='Drug concentration',
    template='plotly_white'
)
fig.update_yaxes(type="log", dtick=1)
fig.show()

In [24]:
# Define log-likelihood
log_likelihood = chi.LogLikelihood(
    mechanistic_model,
    error_model,
    observations=measurements,
    times=measurement_times
)

# Evaluate log-likelihood for made-up parameters
madeup_parameters = [2, 1.4, 7, 3, 0.2]
score_1 = log_likelihood(madeup_parameters)

# Evaluate log-likelihood for data-generating parameters
score_2 = log_likelihood(PK_params)

print("made-up parameters:", score_1)
print("data-generating parameters:", score_2)

INFO:root:running build_ext
INFO:root:building 'myokit_sim_23_3684731453083825083' extension
INFO:root:creating build
INFO:root:creating build/temp.linux-x86_64-cpython-310
INFO:root:creating build/temp.linux-x86_64-cpython-310/tmp
INFO:root:creating build/temp.linux-x86_64-cpython-310/tmp/tmpnem72d_7myokit
INFO:root:x86_64-linux-gnu-gcc -Wno-unused-result -Wsign-compare -DNDEBUG -g -fwrapv -O2 -Wall -g -fstack-protector-strong -Wformat -Werror=format-security -g -fwrapv -O2 -fPIC -I/usr/local/include -I/opt/local/include -I/home/rumney/Documents/Myleotoxicity/Myleotoxicity-PKPD/venv/lib/python3.10/site-packages/myokit/_sim -I/home/rumney/Documents/Myleotoxicity/Myleotoxicity-PKPD/venv/lib/python3.10/site-packages/myokit/_sim -I/home/rumney/Documents/Myleotoxicity/Myleotoxicity-PKPD/venv/include -I/usr/include/python3.10 -c /tmp/tmpnem72d_7myokit/source.c -o build/temp.linux-x86_64-cpython-310/tmp/tmpnem72d_7myokit/source.o
INFO:root:creating build/lib.linux-x86_64-cpython-310
INFO:roo

made-up parameters: -99.13874245105524
data-generating parameters: 140.15851082943192


In [25]:
import pints

# Run optimisation
initial_parameters = [9, 3, 5, 1, 0.5]
parameters_mle, score = pints.optimise(
    log_likelihood, initial_parameters, method=pints.CMAES)

Maximising LogPDF
Using Covariance Matrix Adaptation Evolution Strategy (CMA-ES)
Running in sequential mode.
Population size: 8
Iter. Eval. Best      Time m:s
0     8     -684524.1   0:00.1
1     16    -684524.1   0:00.1
2     24    -296076.3   0:00.1
3     32    -86437.24   0:00.1
20    168    90.09183   0:00.2


/home/rumney/Documents/Myleotoxicity/Myleotoxicity-PKPD/venv/lib/python3.10/site-packages/chi/_error_models.py:1296: RuntimeWarning:

invalid value encountered in log



40    328    98.99385   0:00.3
60    488    108.4932   0:00.3
80    648    116.4927   0:00.4
100   808    138.5995   0:00.5
120   968    140.5558   0:00.6
140   1128   140.5682   0:00.7
160   1288   140.569    0:00.8
180   1448   140.5694   0:00.9
200   1608   140.5694   0:01.0
220   1768   140.5694   0:01.1
240   1928   140.5694   0:01.2
260   2088   140.5694   0:01.3
280   2248   140.5694   0:01.3
300   2408   140.5695   0:01.4
320   2568   140.5695   0:01.5
340   2728   140.5695   0:01.6
360   2888   140.5695   0:01.7
380   3048   140.5695   0:01.8
400   3208   140.5695   0:01.8
420   3368   140.5695   0:01.9
440   3528   140.5695   0:02.0
460   3688   140.5695   0:02.1
480   3848   140.5695   0:02.2
500   4008   140.5695   0:02.3
520   4168   140.5695   0:02.3
540   4328   140.5695   0:02.4
560   4488   140.5695   0:02.5
580   4648   140.5695   0:02.6
600   4808   140.5695   0:02.7
620   4968   140.5695   0:02.8
630   5040   140.5695   0:02.8
Halting: No significant change for 200 

In [26]:
print("MLE parameters:", parameters_mle)
print("Loglikelihood:", score)

MLE parameters: [2.74747846 0.64945759 2.184695   4.75852167 0.08419209]
Loglikelihood: 140.5695249759059


In [27]:
mle_output = mechanistic_model.simulate(parameters_mle, times)
fig.add_trace(go.Scatter(
    x=times,
    y=mle_output[0],
    mode='lines',
    line_dash='dash',
    name='Inferred model'
))
fig.show()

In [8]:
# Define log-posterior
log_prior = pints.ComposedLogPrior(
    pints.UniformLogPrior(0, 20),  # Initial drug amount
    pints.UniformLogPrior(0, 20),  # Compartment volume
    pints.UniformLogPrior(0, 20),  # Elimination rate
    pints.UniformLogPrior(0, 20)   # Sigma
)
log_posterior = chi.LogPosterior(log_likelihood, log_prior)

# Evaluate log-posterior
score_1 = log_posterior(madeup_parameters)
score_2 = log_posterior(true_parameters)
print("made-up parameters:", score_1)
print("data-generating parameters:", score_2)

made-up parameters: -98.1250784620662
data-generating parameters: -1.6582553629286547


In [9]:
# Run inference
controller = chi.SamplingController(log_posterior)
controller.set_sampler(pints.HaarioBardenetACMC)
n_iterations = 5000
posterior_samples = controller.run(n_iterations)

In [12]:
from plotly.subplots import make_subplots

# Discard warmup iterations
warmup = 3000  # This number is not arbitrary but was carefully chosen
posterior_samples = posterior_samples.sel(draw=slice(warmup, n_iterations))

# Visualise posteriors
fig = make_subplots(rows=2, cols=2, shared_yaxes=True)
fig.add_trace(
    go.Histogram(
        name='Posterior samples',
        x=posterior_samples['central.drug_amount'].values.flatten(),
        histnorm='probability',
        showlegend=False
    ),
    row=1,
    col=1
)
fig.add_trace(
    go.Scatter(
        name='Data-generating parameters',
        x=[true_parameters[0]]*2,
        y=[0, 0.04],
        mode='lines',
        line_color='black',
        showlegend=False
    ),
    row=1,
    col=1
)

fig.add_trace(
    go.Histogram(
        name='Posterior samples',
        x=posterior_samples['central.K_cl'].values.flatten(),
        histnorm='probability',
        showlegend=False
    ),
    row=1,
    col=2
)
fig.add_trace(
    go.Scatter(
        name='Data-generating parameters',
        x=[true_parameters[1]]*2,
        y=[0, 0.04],
        mode='lines',
        line_color='black',
        showlegend=False
    ),
    row=1,
    col=2
)

fig.add_trace(
    go.Histogram(
        name='Posterior samples',
        x=posterior_samples['central.V_c'].values.flatten(),
        histnorm='probability',
        showlegend=False
    ),
    row=2,
    col=1
)
fig.add_trace(
    go.Scatter(
        name='Data-generating parameters',
        x=[true_parameters[2]]*2,
        y=[0, 0.04],
        mode='lines',
        line_color='black',
        showlegend=False
    ),
    row=2,
    col=1
)

fig.add_trace(
    go.Histogram(
        name='Posterior samples',
        x=posterior_samples['Sigma'].values.flatten(),
        histnorm='probability',
        showlegend=False
    ),
    row=2,
    col=2
)
fig.add_trace(
    go.Scatter(
        name='Data-generating parameters',
        x=[true_parameters[3]]*2,
        y=[0, 0.04],
        mode='lines',
        line_color='black',
        showlegend=False
    ),
    row=2,
    col=2
)

fig.update_layout(
    xaxis_title='Initial drug amount',
    xaxis2_title='Elimination rate',
    xaxis3_title='Compartment volume',
    xaxis4_title='Sigma',
    yaxis_title='Probability',
    yaxis3_title='Probability',
    template='plotly_white'
)
fig.show()

In [13]:
import plotly.colors

# Set different dosing regimen (for illustration)
mechanistic_model.set_dosing_regimen(dose=5, start=3, period=1, duration=0.3)

# Define patient parameters
parameters_patient_1 = [
    10,  # Initial drug amount
    1,   # Elimination rate
    2,   # Volume of the compartment
]
parameters_patient_2 = [
    10,  # Initial drug amount
    0.5, # Elimination rate
    2,   # Volume of the compartment
]
parameters_patient_3 = [
    10,  # Initial drug amount
    0.8, # Elimination rate
    2,   # Volume of the compartment
]
error_model_params = [0.4]

# Measure drug concentrations of patients
n_times = 1000
times = np.linspace(start=0, stop=10, num=n_times)
measurement_times = times[::50]
result_1 = mechanistic_model.simulate(parameters_patient_1, times)[0]
measurements_1 = error_model.sample(
    error_model_params, result_1[::50], seed=41)[:, 0]
result_2 = mechanistic_model.simulate(parameters_patient_2, times)[0]
measurements_2 = error_model.sample(
    error_model_params, result_2[::50], seed=42)[:, 0]
result_3 = mechanistic_model.simulate(parameters_patient_3, times)[0]
measurements_3 = error_model.sample(
    error_model_params, result_3[::50], seed=43)[:, 0]

# Visualise results
fig = go.Figure()
colors = plotly.colors.qualitative.Plotly
fig.add_trace(go.Scatter(
    x=times,
    y=result_1,
    mode='lines',
    line_color=colors[0],
    name='Model patient 1'
))
fig.add_trace(go.Scatter(
    x=times,
    y=result_2,
    mode='lines',
    line_color=colors[1],
    name='Model patient 2'
))
fig.add_trace(go.Scatter(
    x=times,
    y=result_3,
    mode='lines',
    line_color=colors[2],
    name='Model patient 3'
))
fig.add_trace(go.Scatter(
    x=measurement_times,
    y=measurements_1,
    mode='markers',
    marker_color=colors[0],
    name='Meas. patient 1'
))
fig.add_trace(go.Scatter(
    x=measurement_times,
    y=measurements_2,
    mode='markers',
    marker_color=colors[1],
    name='Meas. patient 2'
))
fig.add_trace(go.Scatter(
    x=measurement_times,
    y=measurements_3,
    mode='markers',
    marker_color=colors[2],
    name='Meas. patient 3'
))
fig.update_layout(
    xaxis_title='Time',
    yaxis_title='Drug concentration',
    template='plotly_white'
)
fig.show()

In [16]:
# Define population model
population_model = chi.ComposedPopulationModel([
    chi.PooledModel(dim_names=['Initial drug amount']),
    chi.LogNormalModel(dim_names=['Elimination rate']),
    chi.PooledModel(dim_names=['Compartment volume']),
    chi.PooledModel(dim_names=['Sigma'])
])

# Define population parameters
population_parameters = [
    10,     # Pooled initial drug amount
    -0.55,  # Log mean elimination rate (mu k_e)
    0.3,    # Log std. elimination rate (sigma k_e)
    2,      # Pooled compartment volume
    0.4     # Pooled measurement noise
]

# Sample individuals from population model
n_ids = 1000
individual_parameters = population_model.sample(
    parameters=population_parameters,
    n_samples=n_ids,
    seed=1
)

# Visualise population model (parameter space)
fig = make_subplots(rows=2, cols=2)
fig.add_trace(
    go.Histogram(
        name='Pop. model samples',
        x=individual_parameters[:, 0],
        histnorm='probability',
        showlegend=False,
        xbins_size=0.01,
        marker_color='lightgrey'
    ),
    row=1,
    col=1
)
fig.add_trace(
    go.Histogram(
        name='Pop. model samples',
        x=individual_parameters[:, 1],
        histnorm='probability',
        showlegend=False,
        marker_color='lightgrey'
    ),
    row=1,
    col=2
)
fig.add_trace(
    go.Histogram(
        name='Pop. model samples',
        x=individual_parameters[:, 2],
        histnorm='probability',
        showlegend=False,
        xbins_size=0.01,
        marker_color='lightgrey'
    ),
    row=2,
    col=1
)
fig.add_trace(
    go.Histogram(
        name='Pop. model samples',
        x=individual_parameters[:, 3],
        histnorm='probability',
        showlegend=False,
        xbins_size=0.01,
        marker_color='lightgrey'
    ),
    row=2,
    col=2
)
fig.add_trace(
    go.Scatter(
        name='Patient 1',
        x=[parameters_patient_1[1], parameters_patient_1[1]],
        y=[0, 0.12],
        mode='lines',
        line=dict(color=colors[0], dash='dash'),
        showlegend=False
    ),
    row=1,
    col=2
)
fig.add_trace(
    go.Scatter(
        name='Patient 2',
        x=[parameters_patient_2[1], parameters_patient_2[1]],
        y=[0, 0.12],
        mode='lines',
        line=dict(color=colors[1], dash='dash'),
        showlegend=False
    ),
    row=1,
    col=2
)
fig.add_trace(
    go.Scatter(
        name='Patient 3',
        x=[parameters_patient_3[1], parameters_patient_3[1]],
        y=[0, 0.12],
        mode='lines',
        line=dict(color=colors[2], dash='dash'),
        showlegend=False
    ),
    row=1,
    col=2
)
fig.update_layout(
    xaxis_title='Initial drug amount',
    xaxis2_title='Elimination rate',
    xaxis3_title='Compartment volume',
    xaxis4_title='Sigma',
    yaxis_title='Probability',
    yaxis3_title='Probability',
    xaxis_range=[9.8, 10.2],
    xaxis3_range=[1.8, 2.2],
    xaxis4_range=[0.2, 0.6],
    template='plotly_white'
)
fig.show()

In [17]:
# Simulate population distribution of measurements
pop_measurements = np.empty(shape=(n_ids, n_times))
for idd, patient_parameters in enumerate(individual_parameters):
    result = mechanistic_model.simulate(patient_parameters[:-1], times)[0]
    pop_measurements[idd] = error_model.sample(
        patient_parameters[-1:], result)[:, 0]

# Visualise population model (measurement space)
fig = go.Figure()

# Plot 5th to 95th percentile of population distribution
fifth = np.percentile(pop_measurements, q=5, axis=0)
ninety_fifth = np.percentile(pop_measurements, q=95, axis=0)
fig.add_trace(go.Scatter(
    x=np.hstack([times, times[::-1]]),
    y=np.hstack([fifth, ninety_fifth[::-1]]),
    line=dict(width=1, color='lightgrey'),
    fill='toself',
    name='Population model',
    text=r"90% bulk probability",
    hoverinfo='text',
    showlegend=True
))

# Plot patient measurements
fig.add_trace(go.Scatter(
    x=measurement_times,
    y=measurements_1,
    mode='markers',
    marker_color=colors[0],
    name='Meas. patient 1'
))
fig.add_trace(go.Scatter(
    x=measurement_times,
    y=measurements_2,
    mode='markers',
    marker_color=colors[1],
    name='Meas. patient 2'
))
fig.add_trace(go.Scatter(
    x=measurement_times,
    y=measurements_3,
    mode='markers',
    marker_color=colors[2],
    name='Meas. patient 3'
))
fig.update_layout(
    xaxis_title='Time',
    yaxis_title='Drug concentration',
    template='plotly_white'
)
fig.show()

In [18]:
# Define individual log-likelihoods for patients
log_likelihood_1 = chi.LogLikelihood(
    mechanistic_model, error_model, measurements_1, measurement_times)
log_likelihood_2 = chi.LogLikelihood(
    mechanistic_model, error_model, measurements_2, measurement_times)
log_likelihood_3 = chi.LogLikelihood(
    mechanistic_model, error_model, measurements_3, measurement_times)

# Define hierarchical log-likelihood
log_likelihood = chi.HierarchicalLogLikelihood(
    log_likelihoods=[log_likelihood_1, log_likelihood_2, log_likelihood_3],
    population_model=population_model)

INFO:root:running build_ext
INFO:root:building 'myokit_sim_4_4205179654949782451' extension
INFO:root:creating build
INFO:root:creating build/temp.linux-x86_64-cpython-310
INFO:root:creating build/temp.linux-x86_64-cpython-310/tmp
INFO:root:creating build/temp.linux-x86_64-cpython-310/tmp/tmps4y7t0cvmyokit
INFO:root:x86_64-linux-gnu-gcc -Wno-unused-result -Wsign-compare -DNDEBUG -g -fwrapv -O2 -Wall -g -fstack-protector-strong -Wformat -Werror=format-security -g -fwrapv -O2 -fPIC -I/usr/local/include -I/opt/local/include -I/home/rumney/Documents/Myleotoxicity/Myleotoxicity-PKPD/venv/lib/python3.10/site-packages/myokit/_sim -I/home/rumney/Documents/Myleotoxicity/Myleotoxicity-PKPD/venv/lib/python3.10/site-packages/myokit/_sim -I/home/rumney/Documents/Myleotoxicity/Myleotoxicity-PKPD/venv/include -I/usr/include/python3.10 -c /tmp/tmps4y7t0cvmyokit/source.c -o build/temp.linux-x86_64-cpython-310/tmp/tmps4y7t0cvmyokit/source.o
INFO:root:creating build/lib.linux-x86_64-cpython-310
INFO:root

In [21]:
# Define hierarchical log-posterior
log_prior = pints.ComposedLogPrior(
    pints.LogNormalLogPrior(1, 1),      # Initial drug amount
    pints.GaussianLogPrior(-1, 1),      # Log mean elimination rate
    pints.LogNormalLogPrior(-1, 0.4),   # Log std. elimination rate
    pints.LogNormalLogPrior(1, 1),      # Compartment volume
    pints.LogNormalLogPrior(1, 1)       # Sigma
)
log_posterior = chi.HierarchicalLogPosterior(
    log_likelihood=log_likelihood, log_prior=log_prior)

# Infer posterior
controller = chi.SamplingController(log_posterior, seed=42)
controller.set_n_runs(1)
controller.set_parallel_evaluation(False)
controller.set_sampler(pints.NoUTurnMCMC)
n_iterations = 2000
posterior_samples = controller.run(n_iterations, log_to_screen=True)

# Discard warmup iterations
warmup = 500  # This number is not arbitrary but was carefully chosen
posterior_samples = posterior_samples.sel(draw=slice(warmup, n_iterations))

Using No-U-Turn MCMC
Generating 1 chains.
Running in sequential mode.
Iter. Eval. Accept.   Steps. Time m:s
0     7      0.167    1        0:00.0
1     8      0.25     1        0:00.0
2     9      0.25     1        0:00.0
3     11     0.209    1        0:00.0


/home/rumney/Documents/Myleotoxicity/Myleotoxicity-PKPD/venv/lib/python3.10/site-packages/chi/_population_models.py:2296: RuntimeWarning:

invalid value encountered in log

/home/rumney/Documents/Myleotoxicity/Myleotoxicity-PKPD/venv/lib/python3.10/site-packages/chi/_population_models.py:2297: RuntimeWarning:

invalid value encountered in log

/home/rumney/Documents/Myleotoxicity/Myleotoxicity-PKPD/venv/lib/python3.10/site-packages/chi/_population_models.py:2382: RuntimeWarning:

invalid value encountered in log

/home/rumney/Documents/Myleotoxicity/Myleotoxicity-PKPD/venv/lib/python3.10/site-packages/chi/_population_models.py:2383: RuntimeWarning:

invalid value encountered in log

/home/rumney/Documents/Myleotoxicity/Myleotoxicity-PKPD/venv/lib/python3.10/site-packages/chi/_population_models.py:2384: RuntimeWarning:

invalid value encountered in log



20    643    0.729    36       0:02.7
40    3569   0.754661 140      0:13.9
60    4939   0.724    66       0:22.7
80    5959   0.769    49       0:29.1
100   6718   0.744    36       0:33.7
120   7162   0.705    22       0:36.5
140   7842   0.771    33       0:40.9
160   8185   0.737    17       0:42.9
180   8570   0.757    19       0:45.2
200   8850   0.774    14       0:46.9
220   9196   0.736    17       0:49.0
240   9450   0.751    13       0:50.5
260   9712   0.746    13       0:52.1
280   10134  0.753    21       0:54.6
300   10476  0.748    17       0:56.7
320   10710  0.764    12       0:58.1
340   10998  0.757    14       0:59.9
360   11228  0.754    11       1:01.3
380   11446  0.773    11       1:02.6
400   11641  0.75     10       1:03.8
420   11909  0.753    13       1:05.6
440   12201  0.759    14       1:07.4
460   12451  0.718    12       1:08.9
480   12791  0.745    17       1:10.9
500   13093  0.744    15       1:12.7
520   13357  0.835    13       1:14.3
540   13657 

In [23]:
# Visualise posteriors
fig = make_subplots(rows=2, cols=2, shared_yaxes=True)
fig.add_trace(
    go.Histogram(
        name='Posterior samples',
        x=posterior_samples['Pooled Initial drug amount'].values.flatten(),
        histnorm='probability',
        marker_color='lightgrey',
        showlegend=False
    ),
    row=1,
    col=1
)
fig.add_trace(
    go.Scatter(
        name='Data-generating parameters',
        x=[population_parameters[0]]*2,
        y=[0, 0.08],
        mode='lines',
        line_color='black',
        showlegend=False
    ),
    row=1,
    col=1
)

fig.add_trace(
    go.Histogram(
        name='Patient 1 Post. samples',
        x=posterior_samples['central.K_cl'].values[
            0, :, 0].flatten(),
        histnorm='probability',
        showlegend=False,
        marker=dict(color=colors[0], opacity=0.7)
    ),
    row=1,
    col=2
)
fig.add_trace(
    go.Histogram(
        name='Patient 1 Post. samples',
        x=posterior_samples['central.K_cl'].values[
            0, :, 1].flatten(),
        histnorm='probability',
        showlegend=False,
        marker=dict(color=colors[1], opacity=0.7)
    ),
    row=1,
    col=2
)
fig.add_trace(
    go.Histogram(
        name='Patient 1 post. samples',
        x=posterior_samples['central.K_cl'].values[
            0, :, 2].flatten(),
        histnorm='probability',
        showlegend=False,
        marker=dict(color=colors[2], opacity=0.7)
    ),
    row=1,
    col=2
)
fig.add_trace(
    go.Scatter(
        name='Patent 1 data-gen. parameters',
        x=[parameters_patient_1[1]]*2,
        y=[0, 0.06],
        mode='lines',
        line_color='darkblue',
        showlegend=False
    ),
    row=1,
    col=2
)
fig.add_trace(
    go.Scatter(
        name='Patent 2 data-gen. parameters',
        x=[parameters_patient_2[1]]*2,
        y=[0, 0.06],
        mode='lines',
        line_color='darkred',
        showlegend=False
    ),
    row=1,
    col=2
)
fig.add_trace(
    go.Scatter(
        name='Patent 3 data-gen. parameters',
        x=[parameters_patient_3[1]]*2,
        y=[0, 0.06],
        mode='lines',
        line_color='darkgreen',
        showlegend=False
    ),
    row=1,
    col=2
)

fig.add_trace(
    go.Histogram(
        name='Posterior samples',
        x=posterior_samples['Pooled Compartment volume'].values.flatten(),
        histnorm='probability',
        marker_color='lightgrey',
        showlegend=False
    ),
    row=2,
    col=1
)
fig.add_trace(
    go.Scatter(
        name='Data-generating parameters',
        x=[population_parameters[3]]*2,
        y=[0, 0.08],
        mode='lines',
        line_color='black',
        showlegend=False
    ),
    row=2,
    col=1
)

fig.add_trace(
    go.Histogram(
        name='Posterior samples',
        x=posterior_samples['Pooled Sigma'].values.flatten(),
        histnorm='probability',
        marker_color='lightgrey',
        showlegend=False
    ),
    row=2,
    col=2
)
fig.add_trace(
    go.Scatter(
        name='Data-generating parameters',
        x=[population_parameters[4]]*2,
        y=[0, 0.06],
        mode='lines',
        line_color='black',
        showlegend=False
    ),
    row=2,
    col=2
)

fig.update_layout(
    xaxis_title='Initial drug amount',
    xaxis2_title='Elimination rate',
    xaxis3_title='Compartment volume',
    xaxis4_title='Sigma',
    yaxis_title='Probability',
    yaxis3_title='Probability',
    template='plotly_white',
    bargap=0,
    bargroupgap=0,
    barmode='overlay'
)
fig.show()

In [24]:
fig = make_subplots(rows=1, cols=2, shared_yaxes=True)
fig.add_trace(
    go.Histogram(
        name='Posterior samples',
        x=posterior_samples['Log mean Elimination rate'].values.flatten(),
        histnorm='probability',
        marker_color='lightgrey',
        showlegend=False
    ),
    row=1,
    col=1
)
fig.add_trace(
    go.Scatter(
        name='Data-generating parameters',
        x=[population_parameters[1]]*2,
        y=[0, 0.08],
        mode='lines',
        line_color='black',
        showlegend=False
    ),
    row=1,
    col=1
)

fig.add_trace(
    go.Histogram(
        name='Posterior samples',
        x=posterior_samples['Log std. Elimination rate'].values.flatten(),
        histnorm='probability',
        marker_color='lightgrey',
        showlegend=False
    ),
    row=1,
    col=2
)
fig.add_trace(
    go.Scatter(
        name='Data-generating parameters',
        x=[population_parameters[2]]*2,
        y=[0, 0.08],
        mode='lines',
        line_color='black',
        showlegend=False
    ),
    row=1,
    col=2
)

fig.update_layout(
    xaxis_title='Log mean elimination rate',
    xaxis2_title='Log std. elimination rate',
    yaxis_title='Probability',
    template='plotly_white'
)
fig.show()